<a href="https://colab.research.google.com/github/Shahabshms/Numerical_Methods_for_ML_and_AI_Solution_3/blob/main/_4301_HW3_Q2_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implement the interior point method in Python for the above problem. Your function should take as input the initial guess $x^{(0)}$, $\delta^{(0)}$, $\rho$, the number of iterations $t$ of the interior point method to perform, and a threshold $\epsilon > 0$ for the convergence of Newton’s method. It should return the vector $x^{(t)}$. For Newton’s method, you should terminate if $.5(x^*)^T H(x^{(t)})x^* \leq \epsilon$, where $x^*$ is the solution to the Newton subproblem at $x^{(t)}$, see Slide 7 33.

Define $h(\lambda) = \frac{\delta}{2}\Vert q - \sum_{m=1}^M \lambda_m x^{(m)}\Vert_2^2 - \sum_{m=1}^M\log(\lambda_m)$. 





Now
\begin{align*}
    \frac{\partial h(\lambda)}{\partial \lambda_k} = \frac{\delta}{2} \sum_{i=1}^n 2(q_i - \sum_{m=1}^M \lambda_{m}x_i^{(m)})(-x_i^{k}) - \frac{1}{\lambda_k}
\end{align*}
\begin{align*}
    \nabla_\lambda h(\lambda) = \left[ -\delta\sum_{i=1}^n x_i^{(1)}(q_i - \sum_{m=1}^M \lambda_mx_i^{(m)}) -\frac{1}{\lambda_1};\dots; -\delta\sum_{i=1}^n x_i^{(M)}(q_i - \sum_{m=1}^M \lambda_mx_i^{(m)}) -\frac{1}{\lambda_M} \right]
\end{align*}

In [ ]:
def get_gradient(lambd_a,delta):

  gradient = np.zeros([M,1])
  for j in range(M):
    sum_n = 0
    for i in range(n):
      sum_m = 0
      for m in range(M):
        sum_m += lambd_a[m]*x[m][i]

      sum_n += x[j][i] * (q[i] - sum_m)

    gradient[j] = - delta * sum_n - 1/lambd_a[j]

  return gradient

\begin{align*}
    H_h(\lambda) = \left[\begin{array}{ccc}
         \delta\sum_{i=1}^n {x_i^{(1)}}^2 + \cfrac{1}{\lambda_1^2}&\dots&\delta\sum_{i=1}^n x_i^{(1)}x^{(M)}_i \\
         \vdots&\ddots&\vdots\\
         \delta\sum_{i=1}^n x_i^{(M)}x^{(1)}_i&\dots&\delta\sum_{i=1}^n {x_i^{(M)}}^2 + \cfrac{1}{\lambda_M^2}
    \end{array}\right]
\end{align*}

In [ ]:
def get_hessian(lambd_a,delta):
  hessian = np.zeros([M,M])
  for i in range(M):
    for j in range(M):
      if i == j:
        hessian[i][j] = delta * np.matmul(x[i],np.transpose(x[j])) + 1/(lambd_a[i]**2)
      else:
        hessian[i][j] = delta * np.matmul(x[i],np.transpose(x[j]))

  return hessian

And since this is a constrained problem 
\begin{align*}
\left[\begin{array}{cc}
         H_h(\lambda_t)&A^T \\
         A&0
    \end{array}\right] \left[\begin{array}{c}
         \lambda^*  \\
         \nu^*
    \end{array}\right] = \left[\begin{array}{c}
         -\nabla_\lambda h(\lambda_t)\\
         0
    \end{array}\right]
\end{align*}

At each step of the Newton method, the update is 
\begin{align*}
    \lambda^{(t+1)} = \lambda^{(t)} + \gamma\lambda^* 
\end{align*}


In [ ]:
def Newton_method(lambd_a,delta,epsilon):

  while True:

    gamma = 0.05

    geradient = get_gradient(lambd_a,delta)
    hessian = get_hessian(lambd_a,delta)

    left_hand_side_mat = np.block([[hessian,np.transpose(A)],[A,0]])
    right_hand_side_mat = np.block([[-geradient],[0]])
    ans = np.linalg.solve(left_hand_side_mat,right_hand_side_mat)
    lambda_star = ans[:-1]
    nu_star = ans[-1]

    lambd_a = lambd_a + gamma * lambda_star

    if 0.5 * np.matmul(np.matmul(np.transpose(lambda_star),hessian),lambda_star) < epsilon:
      break

  return lambd_a

Then the steps of the interior points are :
$$\lambda^* \in \text{argmin}_{\lambda\in\mathbb{R}^n\text{ s.t. } A\lambda=b}\left[ h(\lambda )\right] $$
$$ \lambda^{(t+1)} = \lambda^* $$
$$\delta^{(t+1)} = \rho \delta^{(t)}$$

In [ ]:
def interior_point(initial_lambda,rho,delta,epsilon):

  lambd_a = initial_lambda
  for iteration in range(max_iterations):

    lambd_a = Newton_method(lambd_a,delta,epsilon)
    delta = rho * delta

  return lambd_a

In [ ]:
import numpy as np
import math

n = 2
M = 4
x = np.array([[-1,-1],[-1,1],[1,-1],[1,1]])
q = np.array([1.7,1.2])
A = np.ones([1,M])
b = 1
initial_lambda = np.ones([M,1])/M
max_iterations = 200
rho = 1.2
delta = 1.2
epsilon = 0.5

final_lambda = interior_point(initial_lambda,rho,delta,epsilon)

print(final_lambda)

point = np.zeros(n)
for m in range(M):
  point+= final_lambda[m] * x[m]

print(point)


[[1.23464627e-16]
 [1.58740235e-16]
 [5.55590823e-16]
 [1.00000000e+00]]
[1. 1.]
